In [1]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import os
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.news_db
collection = db.articles

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Url to scrape
url = 'https://redplanetscience.com/'
browser.visit(url)

In [6]:
titles = []
previews = []

post = {}

# Iterate through the landing page:
for x in range(1):
    html = browser.html
    soup = bs(html, 'lxml')
    
    news_title = soup.find_all('div', class_='content_title')
    news_preview = soup.find_all('div', class_='article_teaser_body')
    
    for title in news_title:
        n_title = title.text
        titles.append(n_title)
        
    for preview in news_preview:
        n_prev = preview.text
        previews.append(n_prev)
        
    # Zip List Together and print it 
    titles_and_prevs = zip(titles, previews)
    print(list(titles_and_prevs))
        
    # Dictionary to be inserted into MongoDB
    post = {
        'title': titles,
        'preview': previews,
    }
    # Insert in Mongo DB Database
    collection.insert_one(post)

[("How NASA's Mars Helicopter Will Reach the Red Planet's Surface", 'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'), ("Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest", "Nine finalists have been chosen in the essay contest for K-12 students across U.S. to name NASA's next Mars rover. Now you can help by voting for your favorite. "), ("NASA-JPL Names 'Rolling Stones Rock' on Mars", "NASA's Mars InSight mission honored one of the biggest bands of all time at Pasadena concert."), ('NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities', 'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'), ("NASA's Curiosity Keeps Rolling As Team Operates Rover From Home", 'The team has learned to meet new challenges as they work remotel

In [7]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)

{'_id': ObjectId('632103790ffbff62bb6e4a07'), 'title': ["How NASA's Mars Helicopter Will Reach the Red Planet's Surface", "Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest", "NASA-JPL Names 'Rolling Stones Rock' on Mars", 'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities', "NASA's Curiosity Keeps Rolling As Team Operates Rover From Home", 'Join NASA for the Launch of the Mars 2020 Perseverance Rover', 'The Man Who Wanted to Fly on Mars', "NASA's MAVEN Explores Mars to Understand Radio Interference at Earth", 'NASA Perseverance Mars Rover Scientists Train in the Nevada Desert', "NASA's Mars 2020 Rover Closer to Getting Its Name", 'NASA Moves Forward With Campaign to Return Mars Samples to Earth', "10.9 Million Names Now Aboard NASA's Perseverance Mars Rover", "Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch", "NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound", "NASA InSight's 'Mole' Is Out of Sight"], 'pr

In [8]:
browser.quit()